In [3]:
import video_processing as vp

### Define some static global variables

In [ ]:
WIDTH = 256
HEIGHT = 256
TRUE_FRAMERATE = 28.815
BEHAVIOUR_FRAMERATE = 1/(11090.9/1e6)

In [ ]:
l_mouse = 
r_mouse = 

### Extract the color channels from the RAW files

In [ ]:
l_mouse_green_frames = vp.extract_channel(
    filename=l_mouse,
    channel='green',
    width=WIDTH,
    height=HEIGHT
)

r_mouse_green_frames = vp.extract_channel(
    filename=r_mouse,
    channel='green',
    width=WIDTH,
    height=HEIGHT
)

l_mouse_blue_frames = vp.extract_channel(
    filename=l_mouse,
    channel='blue',
    width=WIDTH,
    height=HEIGHT
)

r_mouse_blue_frames = vp.extract_channel(
    filename=r_mouse,
    channel='blue',
    width=WIDTH,
    height=HEIGHT
)

### Import RAW timestamps and clean the data

In [ ]:
l_mouse_timestamps = vp.clean_raw_timestamps(l_mouse_timestamps_raw)
r_mouse_timestamps = vp.clean_raw_timestamps(r_mouse_timestamps_raw)

### Remove dark frames

In [ ]:
# slice by threshold on green frames
l_mouse_slice = vp.dark_frames_slice(frames=l_mouse_green_frames)
r_mouse_slice = vp.dark_frames_slice(frames=r_mouse_green_frames)

# slice green frames
l_mouse_green_frames = l_mouse_green_frames[l_mouse_slice]
r_mouse_green_frames = r_mouse_green_frames[r_mouse_slice]

# slice blue frames
l_mouse_blue_frames = l_mouse_blue_frames[l_mouse_slice]
r_mouse_blue_frames = r_mouse_blue_frames[r_mouse_slice]

# slice timestamps
l_mouse_timestamps = l_mouse_timestamps[l_mouse_slice]
r_mouse_timestamps = r_mouse_timestamps[r_mouse_slice]

### Find locations of any dropped frames

In [ ]:
l_mouse_delta, l_mouse_where_dropped = vp.get_locations_of_dropped_frames(
    timestamps=l_mouse_timestamps,
    fps=30
)

r_mouse_delta, r_mouse_where_dropped = vp.get_locations_of_dropped_frames(
    timestamps=r_mouse_timestamps,
    fps=30
)

### Fill in dropped frames by interpolating available data

In [ ]:
# Create a list of DroppedFrames objects
list_of_dropped_frames = [
    vp.DroppedFrames(*args) for args in vp.generate_frames(
        frames,
        differences,
        locations,
        true_frame_rate
    )  
] # args need clarification !!!

# Produce the interpolated frames
list_of_interpolated_dropped_frames = [
    DroppedFrames.interpolate() for DroppedFrames in list_of_dropped_frames
]

filled_frames = vp.insert_interpolated_frames(
    frames,
    list_of_interpolated_dropped_frames
)

## Calculate $\frac{df}{d\theta}$

In [ ]:
l_mouse_green, l_mouse_green_variance = vp.calculate_df_f0(l_mouse_green)
l_mouse_blue, l_mouse_blue_variance = vp.calculate_df_f0(l_mouse_blue)
l_mouse_red, l_mouse_red_variance = vp.calculate_df_f0(l_mouse_red)

## Apply lfilter using chebychev type 1 filter parameters 

In [ ]:
# initialise the filtering paramters by creating a Filter object
low_limit = 0.01
high_limit = 12.0

filt = vp.Filter(
    low_freq_cutoff=low_limit,
    high_freq_cutoff=high_limit,
    frame_rate=TRUE_FRAMERATE
)

# interpolate the arrays
l_mouse_green = filt.filter(l_mouse_green)
r_mouse_green = filt.filter(r_mouse_green)
l_mouse_blue = filt.filter(l_mouse_blue)
r_mouse_blue = filt.filter(r_mouse_blue)
l_mouse_red = filt.filter(l_mouse_red)
r_mouse_red = filt.filter(r_mouse_red)

## Ensure the frames have the same dimensions

In [ ]:
if l_mouse_green.shape[0] != r_mouse_green.shape[0]:
    if l_mouse_green.shape[0] < r_mouse_green.shape[0]:
        r_mouse_green = r_mouse_green[:l_mouse_green.shape[0]]
        r_mouse_blue = r_mouse_blue[:l_mouse_blue.shape[0]]
    elif r_mouse_green.shape[0] < l_mouse_green.shape[0]:
        l_mouse_green = l_mouse_green[:r_mouse_green.shape[0]]
        l_mouse_blue = l_mouse_blue[:r_mouse_blue.shape[0]]

assert left_mouse_green_frames.shape[0] == right_mouse_green_frames.shape[0]

## Save processed Frames !!!

## Correct Channel A by Channel B: $\frac{A}{1+B}$